Simon Restrepo
________________________________________________________________________________________________


Utilizar transfer learning para implementar un clasificador de imágenes de animales (21 clases), a partir de las librerías de TensorNets. Las siguientes son las pautas para el desarrollo del algoritmo.

Clonar la base de datos H5 desde el repositorio: https://bitbucket.org/Josepamplona212620/animal_data.git

Utilizar el modelo VGG16 de TensorNets para usar las salidas como base para el clasificador

Usar las salidas de la capa 7 del comando get_middles() como entrada para la siguiente estructura: Conv – Relu - Conv – Relu - Maxpool(stride 2) – Flatten – FC – Relu – FC(salida)

Usar kernels de convolución de 3x3 de 512 canales (stride 1), un learning rate de 0.01 y batches de 16 imágenes.

Debe adjuntar los archivos .py o ipyb comprimidos en una carpeta.

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/T_learning/animal")

In [0]:
!git clone https://bitbucket.org/Josepamplona212620/animal_data.git

Cloning into 'animal_data'...
remote: Counting objects: 8, done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (8/8), done.


In [3]:
!pip install cython
!pip install tensornets

    100% |████████████████████████████████| 593kB 22.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/0c/ae/7a/6d31e83c89c2b02c13e8f08ee8e20abe71670061e057a6058f
Successfully built tensornets


In [4]:
!ls animal_data

test_animals.h5  train_animals.h5


In [5]:
import h5py
import numpy as np

# Leer de una base de datos H5
with h5py.File('animal_data/train_animals.h5','r') as h5data:

    ls=list(h5data.keys())
    print(ls)
    train_data=np.array(h5data.get('train_set_x')[:])
    train_labels=np.array(h5data.get('train_set_y')[:])
print(train_data.shape)    
print(train_labels.shape) 

with h5py.File('animal_data/test_animals.h5','r') as h5data:
    ls=list(h5data.keys())
    print(ls)    
    test_data=np.array(h5data.get('test_set_x')[:])
    test_labels=np.array(h5data.get('test_set_y')[:])
print(test_data.shape)    
print(test_labels.shape)  


['train_set_x', 'train_set_y']
(1680, 224, 224, 3)
(1680, 1)
['test_set_x', 'test_set_y']
(420, 224, 224, 3)
(420, 1)


In [6]:
import h5py
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import tensornets as nets

# Hiperparametros
training_epochs=10
batch_size=16  
learning_rate = 0.01
display_step=1

# Parametros de la red neuronal
n_hidden_1 = 1024 # 1st layer number of neurons
n_input = 215 # data input (feature shape ?,7,7,2048)
n_classes = 21 # total classes

X = tf.placeholder(tf.float32, [None, 224, 224, 3])
X2 = tf.placeholder(tf.float32, [None, 14, 14, 512])
Y = tf.placeholder("float", [None, n_classes])

# Cargo el modelo VGG16 de Tensornets, modelo con que se trabajara
model = nets.VGG16(X)
assert isinstance(model, tf.Tensor)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.


In [0]:
# Hago resize a las imagenes para que queden con su arquitectura
def resize_np (np_array):
    resized=[]
    for i in list(np_array):
        larger=cv2.resize(i,(224,224))
        resized.append(np.array(larger))
    return (np.array(resized).astype(np.float32))

# Funcion para cambiar labels a onehot
def one_hot_transformation(labels,n_classes):
    samples=labels.size
    one_hot_labels=np.zeros((samples,n_classes))
    for i in range(samples):
        one_hot_labels[i,labels[i]]=1
    return(one_hot_labels)
  
X_train=resize_np(train_data)
print(X_train.shape)
X_test=resize_np(test_data)
print(X_test.shape)
Y_train=one_hot_transformation(train_labels,n_classes)
print(Y_train.shape)
Y_test=one_hot_transformation(test_labels,n_classes)
print(Y_test.shape)

# Normalizo las imagenes
X_train = X_train/255
X_test = X_test/255


# mostrar un ejemplo
plt.imshow(X_train[20])

In [0]:
# Imprimo el modelo(solo para ver la arquitectura)
model.print_middles()

Scope: vgg16
conv3/1/Relu:0 (?, 56, 56, 256)
conv3/2/Relu:0 (?, 56, 56, 256)
conv3/3/Relu:0 (?, 56, 56, 256)
conv4/1/Relu:0 (?, 28, 28, 512)
conv4/2/Relu:0 (?, 28, 28, 512)
conv4/3/Relu:0 (?, 28, 28, 512)
conv5/1/Relu:0 (?, 14, 14, 512)
conv5/2/Relu:0 (?, 14, 14, 512)
conv5/3/Relu:0 (?, 14, 14, 512)


In [0]:
def initialize_parameters():
  
  tf.set_random_seed(1)

  W1 = tf.get_variable('W1',[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
  W2 = tf.get_variable('W2',[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
  
  
  parameters = {"W1": W1,
                "W2": W2}
  
  return parameters
parameters = initialize_parameters()

In [0]:
# Declaración de los pesos y los bias
weights = {'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev=0.1)),
           
          }
biases  = {'b1': tf.Variable(tf.truncated_normal([n_hidden_1],stddev=0.1)),
          
          }


def multilayer_perceptron(x, parameters):
    
    # Obtención de los pesos desde "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']

    # 1ra CONV2D, padding 'SAME'
    
    Z1 = tf.nn.conv2d(x,W1,strides=[1,1,1,1],padding='SAME')
    
    # RELU 1
    
    A1 = tf.nn.relu(Z1)
    
    # 2da CONV2D, padding 'SAME'
    
    Z2 = tf.nn.conv2d(A1,W2,strides=[1,1,1,1],padding='SAME')
    
    # RELU 2
    
    A2 = tf.nn.relu(Z2)
    
    # MAXPOOL: window 2x2, stride 2 padding 'SAME'
    
    P1 = tf.nn.max_pool(A2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME');
    
    # FLATTEN
    
    F = tf.contrib.layers.flatten(P1)  
    
    # FC con 21 salidas
    Z7 = tf.contrib.layers.fully_connected(F,21,None)
    
    # RELU 3
    A2 = tf.nn.relu(Z7)
    
    # FC Salida OUTPUT
    FOutput = tf.contrib.layers.fully_connected(A2,21,None)
    
    
    return Z7
  
# Declarar la operación que aplica el MLP usando la información de entrada
logits = multilayer_perceptron(X2, parameters)

In [10]:
# Funcion de perdida y optimización 
# para el entrenamiento.
loss_op = tf.losses.softmax_cross_entropy(
    onehot_labels=Y,
    logits=logits,
    weights=1.0,
    scope=None,
    loss_collection=tf.GraphKeys.LOSSES,
    reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS
)
# Optimizador Adam
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing the variables
init = tf.global_variables_initializer()

Instructions for updating:
Use tf.cast instead.


In [0]:
with tf.Session() as sess:
    sess.run(init)
    print("¡Inicio de Pre-entrenamiento!")
    sess.run(model.pretrained())  # equivalent to nets.pretrained(model
    print("¡Finalización de Pre-entrenamiento!")
    for epoch in range(training_epochs):        
        avg_cost = 0.
        #obtiene el numero de grupos en que queda dividida la base de datos
        total_batch = int(Y_train.shape[0]/batch_size) 
        print("Inicio Epoch:",epoch," For barch: ",total_batch)
        
        # ciclo para entrenar con cada grupo de datos
        losses=[]
        for i in range(total_batch-1):
            batch_x= X_train[i*batch_size:(i+1)*batch_size]
            batch_y= Y_train[i*batch_size:(i+1)*batch_size]
            features = model.preprocess(batch_x)
            features = sess.run(model.get_middles(), {X: batch_x})[-1]
            # Correr la funcion de perdida y la operacion de optimización con la respectiva alimentación del placeholder
            _,c =sess.run([train_op, loss_op],feed_dict={X2:features,Y:batch_y})
            # Promedio de resultados de la funcion de pérdida
            losses.append(c)
            avg_cost += c / total_batch
        # Mostrar el resultado del entrenamiento por grupos
        if epoch % display_step == 0:
            print("     Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("¡Optimización  Finalizada!")
 